In [1]:
#Goals of Project: Train a neural network to solve a handwritten math expression.

In [1]:
import numpy as np
import random
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam


In [2]:
!pip install kaggle

In [3]:
from google.colab import files
upload = files.upload()

Saving kaggle.json to kaggle.json


In [4]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
!kaggle datasets download -d xainano/handwrittenmathsymbols

handwrittenmathsymbols.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
import zipfile

with zipfile.ZipFile('handwrittenmathsymbols.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')

In [12]:
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to the handwritten image dataset
handwritten_dataset_path = './dataset/extracted_images'

# Load handwritten images and labels
handwritten_images = []
handwritten_labels = []

for symbol_class in os.listdir(handwritten_dataset_path):
    class_path = os.path.join(handwritten_dataset_path, symbol_class)

    for image_file in os.listdir(class_path):
        image_path = os.path.join(class_path, image_file)

        try:
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            if image is not None:
                image = cv2.resize(image, (64, 64))
                handwritten_images.append(image)
                handwritten_labels.append(symbol_class)
            else:
                print(f"Warning: Unable to read or empty image - {image_path}")

        except Exception as e:
            print(f"Error processing image - {image_path}")
            print(f"Error details: {str(e)}")

# Convert lists to NumPy arrays
handwritten_images = np.array(handwritten_images)
handwritten_labels = np.array(handwritten_labels)

# Encode labels
handwritten_label_encoder = LabelEncoder()
handwritten_labels_encoded = handwritten_label_encoder.fit_transform(handwritten_labels)
handwritten_labels_one_hot = to_categorical(handwritten_labels_encoded)

# Normalize pixel values
handwritten_images = handwritten_images / 255.0

# Split the handwritten dataset into training and testing sets
X_handwritten_train, X_handwritten_test, y_handwritten_train, y_handwritten_test = train_test_split(
    handwritten_images, handwritten_labels_one_hot, test_size=0.2, random_state=42
)

# Reshape the input data for CNN
X_handwritten_train = X_handwritten_train.reshape(X_handwritten_train.shape[0], 64, 64, 1)
X_handwritten_test = X_handwritten_test.reshape(X_handwritten_test.shape[0], 64, 64, 1)

# Print the shapes of the handwritten datasets
print("X_handwritten_train shape:", X_handwritten_train.shape)
print("X_handwritten_test shape:", X_handwritten_test.shape)
print("y_handwritten_train shape:", y_handwritten_train.shape)
print("y_handwritten_test shape:", y_handwritten_test.shape)

# Load MNIST dataset
(X_mnist_train, y_mnist_train), (X_mnist_test, y_mnist_test) = mnist.load_data()

# Resize MNIST images to match the dimensions of the handwritten image dataset (64 by 64)
X_mnist_train_resized = np.array([cv2.resize(image, (64, 64)) for image in X_mnist_train])
X_mnist_test_resized = np.array([cv2.resize(image, (64, 64)) for image in X_mnist_test])

# Reshape and normalize x MNIST data
X_mnist_train_resized = X_mnist_train_resized.reshape(X_mnist_train_resized.shape[0], 64, 64, 1).astype('float32') / 255
X_mnist_test_resized = X_mnist_test_resized.reshape(X_mnist_test_resized.shape[0], 64, 64, 1).astype('float32') / 255

# One-hot encode the labels
y_mnist_train_one_hot = to_categorical(y_mnist_train)
y_mnist_test_one_hot = to_categorical(y_mnist_test)

# Print the shapes of the resized MNIST datasets
print("X_mnist_train_resized shape:", X_mnist_train_resized.shape)
print("X_mnist_test_resized shape:", X_mnist_test_resized.shape)
print("y_mnist_train_one_hot shape:", y_mnist_train_one_hot.shape)
print("y_mnist_test_one_hot shape:", y_mnist_test_one_hot.shape)

# Pad y_handwritten_train and test and y_mnist_train and test to get 17 columns
y_handwritten_train_padded = np.pad(y_handwritten_train, [(0, 0), (0, 10)], mode='constant', constant_values=0)
y_handwritten_test_padded = np.pad(y_handwritten_train, [(0,0), (0,10)], mode='constant', constant_values=0)
y_mnist_train_padded = np.pad(y_mnist_train_one_hot, [(0,0), (0,7)], mode='constant', constant_values=0)
y_mnist_test_padded = np.pad(y_mnist_train_one_hot, [(0,0), (0,7)], mode='constant', constant_values=0)

# Concatenate the handwritten image dataset with the MNIST dataset
X_combined_train = np.concatenate([X_handwritten_train, X_mnist_train_resized], axis=0)
y_combined_train = np.concatenate([y_handwritten_train_padded, y_mnist_train_padded], axis=0)
X_combined_test = np.concatenate([X_handwritten_test, X_mnist_test_resized], axis=0)
y_combined_test = np.concatenate([y_handwritten_test_padded, y_mnist_test_padded], axis=0)

# Ensure consistent sizes for X_combined_test and y_combined_test
X_combined_test = X_combined_test[:len(y_combined_test)]
y_combined_test = y_combined_test[:len(X_combined_test)]

# Print the shapes of the combined datasets
print("X_combined_train shape:", X_combined_train.shape)
print("X_combined_test shape:", X_combined_test.shape)
print("y_combined_train shape:", y_combined_train.shape)
print("y_combined_test shape:", y_combined_test.shape)

# Data augmentation for the datasets
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

X_handwritten_train shape: (77143, 64, 64, 1)
X_handwritten_test shape: (19286, 64, 64, 1)
y_handwritten_train shape: (77143, 7)
y_handwritten_test shape: (19286, 7)
X_mnist_train_resized shape: (60000, 64, 64, 1)
X_mnist_test_resized shape: (10000, 64, 64, 1)
y_mnist_train_one_hot shape: (60000, 10)
y_mnist_test_one_hot shape: (10000, 10)
X_combined_train shape: (137143, 64, 64, 1)
X_combined_test shape: (29286, 64, 64, 1)
y_combined_train shape: (137143, 17)
y_combined_test shape: (29286, 17)


In [13]:
# Define the CNN model
model = Sequential()

# Convolutional layer with 32 filters, a 3x3 kernel, and 'relu' activation function
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)))
# Batch Normalization layer
model.add(BatchNormalization())
# Max pooling layer to reduce spatial dimensions
model.add(MaxPooling2D((2, 2)))

# Another convolutional layer with 64 filters and 'relu' activation
model.add(Conv2D(64, (3, 3), activation='relu'))
# Batch Normalization layer
model.add(BatchNormalization())
# Another max pooling layer
model.add(MaxPooling2D((2, 2)))

# Another convolutional layer with 128 filters and 'relu' activation
model.add(Conv2D(128, (3,3), activation='relu'))
# Batch Normalization layer
model.add(BatchNormalization())
# Another max pooling layer
model.add(MaxPooling2D((2,2)))

# Flatten the output to feed into densely connected layers
model.add(Flatten())

# Dense (fully connected) layer with 128 neurons and 'relu' activation
model.add(Dense(128, activation='relu'))
# Dropout layer to reduce overfitting
model.add(Dropout(0.5))

# Output layer with as many neurons as classes and 'softmax' activation for multi-class classification
model.add(Dense(17, activation='softmax'))

# Compile the model
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model on the training data with validation data
history = model.fit(
    X_combined_train, y_combined_train,
    epochs=5, batch_size=32,
    validation_split=0.2
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_combined_test, y_combined_test, verbose=2)
print(f"\nTest Accuracy: {test_accuracy * 100:.2f}%")

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 62, 62, 32)        320       
                                                                 
 batch_normalization_6 (Bat  (None, 62, 62, 32)        128       
 chNormalization)                                                
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 31, 31, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 batch_normalization_7 (Bat  (None, 29, 29, 64)        256       
 chNormalization)                                                
                                                      

In [30]:
print("X_combined_train shape:", X_combined_train.shape)
print("y_combined_train shape:", y_combined_train.shape)
print("X_combined_test shape:", X_combined_test.shape)
print("y_combined_test shape:", y_combined_test.shape)

X_combined_train shape: (137143, 64, 64, 1)
y_combined_train shape: (137143, 10)
X_combined_test shape: (29286, 64, 64, 1)
y_combined_test shape: (87143, 10)


In [ ]:
#Block to save trained model
from tensorflow.keras.models import model_from_json

# Save model architecture to JSON file
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# Save the entire model as a single HDF5 file
model.save("full_model.h5")

In [ ]:
#Block to load saved trained model
from tensorflow.keras.models import load_model

# Load model architecture from JSON file
with open("model.json", "r") as json_file:
    loaded_model_json = json_file.read()

loaded_model = model_from_json(loaded_model_json)

#Load the entire model from a single HDF5 file
loaded_model = load_model("full_model.h5")